#Google Colaboratory RDP Server

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>

! wget https://raw.githubusercontent.com/VivekDomadia/Google-Colaboratory-RDP-Server/master/RDP.sh &> /dev/null
! chmod +x RDP.sh
! ./RDP.sh